In [6]:
%load_ext autoreload
%autoreload 2

%load_ext lab_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [7]:
import sys
from pathlib import Path

In [8]:
_pwd = Path(".").resolve()
print(_pwd)
sys.path.append(str(_pwd))

/data/ik18445_cache/projects/phenotype-mapping/analysis/notebooks/mvp_ontology_distance


In [9]:
import pandas as pd
import janitor
from pydash import py_

In [10]:
from common_funcs import utils

In [11]:
proj_root = utils.find_project_root("docker-compose.yml")
assert proj_root.exists(), proj_root

data_dir = proj_root / "data"
artifacts_dir = proj_root / "data" / "output" / "mvp-efo-terms-2022-12"
assert artifacts_dir.exists(), artifacts_dir

In [12]:
efo_terms = pd.read_csv(artifacts_dir / "efo_terms.csv")
efo_terms

,efo_id,efo_term,efo_term_clean,id
0,CHEBI_16737,creatinine,creatinine,CHEBI_16737_0
1,CHEBI_16990,bilirubin,bilirubin,CHEBI_16990_1
2,CHEBI_17347,testosterone,testosterone,CHEBI_17347_2
3,CHEBI_23965,estradiol,estradiol,CHEBI_23965_3
4,CHEBI_26020,phosphate,phosphate,CHEBI_26020_4
...,...,...,...,...
11086,UBERON_0000947,aorta,aorta,UBERON_0000947_11086
11087,UBERON_0001467,shoulder,shoulder,UBERON_0001467_11087
11088,UBERON_0002198,neurohypophysis,neurohypophysis,UBERON_0002198_11088
11089,UBERON_0002384,connective tissue,connective tissue,UBERON_0002384_11089


In [32]:
distance = pd.read_csv(artifacts_dir / "distance.csv")
distance

,id_a,id_b,cosine_similarity
0,CHEBI_16737,CHEBI_16990,0.7246
1,CHEBI_16737,CHEBI_17347,0.3148
2,CHEBI_16737,CHEBI_23965,0.1639
3,CHEBI_16737,CHEBI_26020,0.2283
4,CHEBI_16737,CHEBI_35475,0.0281
...,...,...,...
61466323,UBERON_0000045,UBERON_0000947,0.2178
61466324,UBERON_0000045,UBERON_0001467,0.0643
61466325,UBERON_0000310,UBERON_0000947,0.0278
61466326,UBERON_0000310,UBERON_0001467,0.1444


In [35]:
# TERM = "body mass index"
TERM = "alcohol drinking"

bmi_ent = efo_terms[efo_terms["efo_term_clean"] == TERM].reset_index(drop=True)
bmi_ent

,efo_id,efo_term,efo_term_clean,id
0,EFO_0004329,alcohol drinking,alcohol drinking,EFO_0004329_593


In [36]:
ent_id = bmi_ent["efo_id"][0]


subset = distance[
    (distance["id_a"] == ent_id) | (distance["id_b"] == ent_id)
].sort_values("cosine_similarity", ascending=False)[:30]


top_df = pd.DataFrame(
    [
        {
            "id": _["id_a"] if _["id_a"] != ent_id else _["id_b"],
            "score": _["cosine_similarity"],
        }
        for _ in subset.to_dict(orient="records")
    ]
).merge(efo_terms[["efo_id", "efo_term"]].rename(columns={"efo_id": "id"}), on="id")
top_df

,id,score,efo_term
0,EFO_0004315,0.8326,drinking behavior
1,EFO_0007878,0.8218,alcohol consumption measurement
2,EFO_0004777,0.8210,alcohol withdrawal
3,EFO_0004776,0.8008,alcohol and nicotine codependence
4,MONDO_0002046,0.7983,alcohol abuse
5,MONDO_0007079,0.7703,alcohol dependence
6,EFO_0003829,0.7703,alcohol dependence
7,EFO_0007645,0.7488,longitudinal alcohol consumption measurement
8,EFO_0009113,0.7433,alcohol exposure measurement
9,EFO_0004330,0.7140,coffee consumption
